In [2]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Convert depth to meters (assume depth is in millimeters)
depth_img = depth_img.astype(np.float32) / 1000.0

# Get intrinsics
fx, fy = depth_intrinsics["fx"], depth_intrinsics["fy"]
cx, cy = depth_intrinsics["ppx"], depth_intrinsics["ppy"]

# Generate 3D points
height, width = depth_img.shape
points = []
colors = []

for v in range(height):
    for u in range(width):
        z = depth_img[v, u]
        if z == 0:  # Skip invalid depth
            continue

        # Compute 3D point in depth camera frame
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy

        # Apply extrinsic transformation to align with color camera
        point = np.array([x, y, z])
        rotation = np.array(extrinsics["rotation"]).reshape(3, 3)
        translation = np.array(extrinsics["translation"]).reshape(3, 1)
        aligned_point = rotation @ point + translation.flatten()

        points.append(aligned_point)

        # Get color from the color image
        color = color_img[v, u] / 255.0  # Normalize RGB values
        colors.append(color)

# Convert to Open3D point cloud
points = np.array(points)
colors = np.array(colors)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [6]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Convert depth from millimeters to meters
depth_img = depth_img.astype(np.float32) / 1000.0  

# Open3D camera intrinsic setup
color_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    color_intrinsics["width"], color_intrinsics["height"],
    color_intrinsics["fx"], color_intrinsics["fy"],
    color_intrinsics["ppx"], color_intrinsics["ppy"]
)

depth_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    depth_intrinsics["width"], depth_intrinsics["height"],
    depth_intrinsics["fx"], depth_intrinsics["fy"],
    depth_intrinsics["ppx"], depth_intrinsics["ppy"]
)

# Convert images to Open3D format
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))
depth_o3d = o3d.geometry.Image(depth_img)

# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=1.0, depth_trunc=3.0, convert_rgb_to_intensity=False
)

# Create a point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image, color_o3d_intrinsics
)

# Transform point cloud using extrinsic parameters
rotation_matrix = np.array(extrinsics["rotation"]).reshape(3, 3)
translation_vector = np.array(extrinsics["translation"]).reshape(3, 1)

transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_matrix
transformation_matrix[:3, 3] = translation_vector.flatten()

pcd.transform(transformation_matrix)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [8]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Convert depth from millimeters to meters
depth_img = depth_img.astype(np.float32) / 1000.0  

# Open3D camera intrinsic setup
color_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    color_intrinsics["width"], color_intrinsics["height"],
    color_intrinsics["fx"], color_intrinsics["fy"],
    color_intrinsics["ppx"], color_intrinsics["ppy"]
)

depth_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    depth_intrinsics["width"], depth_intrinsics["height"],
    depth_intrinsics["fx"], depth_intrinsics["fy"],
    depth_intrinsics["ppx"], depth_intrinsics["ppy"]
)

# Convert images to Open3D format
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))
depth_o3d = o3d.geometry.Image(depth_img)

# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=1.0, depth_trunc=3.0, convert_rgb_to_intensity=False
)

# Create a point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image, color_o3d_intrinsics
)

# Transform point cloud using extrinsic parameters
rotation_matrix = np.array(extrinsics["rotation"]).reshape(3, 3)
translation_vector = np.array(extrinsics["translation"]).reshape(3, 1)

# Construct a homogeneous transformation matrix
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_matrix
transformation_matrix[:3, 3] = translation_vector.flatten()

# Apply transformation
pcd.transform(transformation_matrix)

# Refine alignment by flipping the axes if needed
flip_transform = np.eye(4)
flip_transform[1, 1] = -1  # Flip Y-axis to correct alignment
pcd.transform(flip_transform)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [ ]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Convert depth to float32 before filtering (Fix for OpenCV error)
depth_img = depth_img.astype(np.float32)
depth_img = cv2.bilateralFilter(depth_img, 9, 75, 75)

# Convert depth from millimeters to meters
depth_img /= 1000.0  

# Open3D camera intrinsic setup
color_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    color_intrinsics["width"], color_intrinsics["height"],
    color_intrinsics["fx"], color_intrinsics["fy"],
    color_intrinsics["ppx"], color_intrinsics["ppy"]
)

# Convert images to Open3D format
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))
depth_o3d = o3d.geometry.Image(depth_img)

# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=1.0, depth_trunc=3.0, convert_rgb_to_intensity=False
)

# Generate point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image, color_o3d_intrinsics
)

# Apply correct transformation using extrinsics
rotation_matrix = np.array(extrinsics["rotation"]).reshape(3, 3)
translation_vector = np.array(extrinsics["translation"]).reshape(3, 1)

transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_matrix
transformation_matrix[:3, 3] = translation_vector.flatten()

# Apply transformation
pcd.transform(transformation_matrix)

# Additional fix: Flip Y-axis if necessary
flip_transform = np.eye(4)
flip_transform[1, 1] = -1
pcd.transform(flip_transform)

# Remove outliers for better visualization
pcd = pcd.voxel_down_sample(voxel_size=0.005)  # Reduces noise
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


/opt/anaconda3/envs/cv-env/lib/python3.13/site-packages/cv2/__init__.py


In [52]:
import numpy as np
import open3d as o3d
import json

# Load camera parameters
with open('camera_params.json', 'r') as file:
    params = json.load(file)

# Camera intrinsics
color_intrinsic = o3d.camera.PinholeCameraIntrinsic(
    width=params['color_intrinsics']['width'],
    height=params['color_intrinsics']['height'],
    fx=params['color_intrinsics']['fx'],
    fy=params['color_intrinsics']['fy'],
    cx=params['color_intrinsics']['ppx'],
    cy=params['color_intrinsics']['ppy']
)

# Correctly load color and depth images
color_raw = o3d.io.read_image(r"/Users/nhuquynhtran/Downloads/Spring2025_Assignment 1/color.png")
depth_raw = o3d.io.read_image(r"/Users/nhuquynhtran/Downloads/Spring2025_Assignment 1/depth.png")

# Verify images loaded correctly
if color_raw is None or depth_raw is None:
    raise FileNotFoundError("Could not load color or depth image. Verify paths and file existence.")

# Extrinsic parameters (depth-to-color)
extrinsic = np.eye(4)
extrinsic[:3, :3] = np.array(params['extrinsics']['rotation']).reshape(3, 3)
extrinsic[:3, 3] = np.array(params['extrinsics']['translation'])

# Create aligned RGB-D image with adjusted parameters
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color=color_raw,
    depth=depth_raw,
    depth_scale=1500.0,  # confirm your depth units (meters or millimeters)
    depth_trunc=5.0,     # set truncation distance to include points
    convert_rgb_to_intensity=False
)

# Generate aligned point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic=color_intrinsic,
    extrinsic=np.linalg.inv(extrinsic)
)


# Flip for correct visualization orientation
pcd.transform([[1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, 1]])

# Visualize the resulting point cloud
o3d.visualization.draw_geometries([pcd])


In [ ]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load color and depth images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Check if images are loaded correctly
if color_img is None or depth_img is None:
    raise FileNotFoundError("Could not load color or depth image. Verify paths and file existence.")

# Convert depth to float32 before filtering (Fix for OpenCV error)
depth_img = depth_img.astype(np.float32)

# Apply bilateral filtering to reduce noise in the depth image
depth_img = cv2.bilateralFilter(depth_img, 9, 75, 75)

# Convert depth from millimeters to meters (if needed)
depth_img /= 1000.0

# Open3D camera intrinsic setup
color_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    color_intrinsics["width"], color_intrinsics["height"],
    color_intrinsics["fx"], color_intrinsics["fy"],
    color_intrinsics["ppx"], color_intrinsics["ppy"]
)

# Convert color and depth images to Open3D format
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
depth_o3d = o3d.geometry.Image(depth_img)

# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color=color_o3d,
    depth=depth_o3d,
    depth_scale=1.0,  # If your depth is in millimeters, use depth_scale=1000.0
    depth_trunc=5.0,  # Set truncation distance to include points closer than 5 meters
    convert_rgb_to_intensity=False
)

# Generate point cloud from the aligned RGB-D image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic=color_o3d_intrinsics
)

# Apply the transformation using the extrinsic parameters
rotation_matrix = np.array(extrinsics["rotation"]).reshape(3, 3)
translation_vector = np.array(extrinsics["translation"]).reshape(3, 1)

# Construct a transformation matrix
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_matrix
transformation_matrix[:3, 3] = translation_vector.flatten()

# Apply transformation to the point cloud
pcd.transform(transformation_matrix)

# Additional fix: Flip for correct visualization orientation
flip_transform = np.eye(4)
flip_transform[1, 1] = -1
pcd.transform(flip_transform)

# Remove outliers for better visualization
pcd = pcd.voxel_down_sample(voxel_size=0.005)  # Reduces noise
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [53]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open("camera_params.json", "r") as f:
    camera_params = json.load(f)

color_intrinsics = camera_params["color_intrinsics"]
depth_intrinsics = camera_params["depth_intrinsics"]
extrinsics = camera_params["extrinsics"]

# Load color and depth images
color_img = cv2.imread("color.png")
depth_img = cv2.imread("depth.png", cv2.IMREAD_UNCHANGED)  # Load depth as 16-bit

# Check if images are loaded correctly
if color_img is None or depth_img is None:
    raise FileNotFoundError("Could not load color or depth image. Verify paths and file existence.")

# Convert depth to float32 before filtering (Fix for OpenCV error)
depth_img = depth_img.astype(np.float32)

# Apply bilateral filtering to reduce noise in the depth image
depth_img = cv2.bilateralFilter(depth_img, 9, 75, 75)

# Convert depth from millimeters to meters (if needed)
depth_img /= 1000.0

# Open3D camera intrinsic setup
color_o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(
    color_intrinsics["width"], color_intrinsics["height"],
    color_intrinsics["fx"], color_intrinsics["fy"],
    color_intrinsics["ppx"], color_intrinsics["ppy"]
)

# Convert color and depth images to Open3D format
color_o3d = o3d.geometry.Image(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
depth_o3d = o3d.geometry.Image(depth_img)

# Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color=color_o3d,
    depth=depth_o3d,
    depth_scale=1.5,  # If your depth is in millimeters, use depth_scale=1000.0
    depth_trunc=5.0,  # Set truncation distance to include points closer than 5 meters
    convert_rgb_to_intensity=False
)

# Generate point cloud from the aligned RGB-D image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic=color_o3d_intrinsics
)

# Apply the transformation using the extrinsic parameters
rotation_matrix = np.array(extrinsics["rotation"]).reshape(3, 3)
translation_vector = np.array(extrinsics["translation"]).reshape(3, 1)

# Construct a transformation matrix
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_matrix
transformation_matrix[:3, 3] = translation_vector.flatten()

# Apply transformation to the point cloud
pcd.transform(transformation_matrix)

# Generate aligned point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic=color_intrinsic,
    extrinsic=np.linalg.inv(extrinsic)
)

# # # Additional fix: Flip for correct visualization orientation
# flip_transform = np.eye(4)
# flip_transform[1, 1] = -1
# pcd.transform(flip_transform)

# Flip for correct visualization orientation
pcd.transform([[1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, 1]])

# # Remove outliers for better visualization
# pcd = pcd.voxel_down_sample(voxel_size=0.005)  # Reduces noise
# pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [15]:
import json
import numpy as np
import cv2
import open3d as o3d

# Load camera parameters
with open('camera_params.json', 'r') as f:
    params = json.load(f)

# Extract parameters with correct key names
color_intrin = params['color_intrinsics']
depth_intrin = params['depth_intrinsics']
R = np.array(params['extrinsics']['rotation']).reshape(3, 3)  # Reshape rotation matrix
T = np.array(params['extrinsics']['translation'])

# Read images
color_image = cv2.imread('color.png')
color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
depth_image = cv2.imread('depth.png', cv2.IMREAD_ANYDEPTH)

# Create depth pixel coordinates
height_d, width_d = depth_image.shape
u_d, v_d = np.meshgrid(np.arange(width_d), np.arange(height_d))
u_d = u_d.reshape(-1)
v_d = v_d.reshape(-1)
depth_values = depth_image.reshape(-1).astype(float)

# Filter invalid depth values
# valid_depth = depth_values > 0
# u_d = u_d[valid_depth]
# v_d = v_d[valid_depth]
# depth_values = depth_values[valid_depth]

# Convert depth to meters (adjust scaling factor if needed)
depth_values /= 1000.0

# Get depth camera parameters
fx_d = depth_intrin['fx']
fy_d = depth_intrin['fy']
cx_d = depth_intrin['ppx']  # Changed from 'cx' to 'ppx'
cy_d = depth_intrin['ppy']  # Changed from 'cy' to 'ppy'

# Calculate 3D coordinates in depth camera space
z = depth_values
x = (u_d - cx_d) * z / fx_d
y = (v_d - cy_d) * z / fy_d
depth_points = np.vstack((x, y, z)).T

# Transform to color camera space
transformed_points = (R @ depth_points.T + T[:, np.newaxis]).T

# Project to color image coordinates
fx_c = color_intrin['fx']
fy_c = color_intrin['fy']
cx_c = color_intrin['ppx']
cy_c = color_intrin['ppy']

x_c = transformed_points[:, 0]
y_c = transformed_points[:, 1]
z_c = transformed_points[:, 2]

u_c = (x_c / z_c) * fx_c + cx_c
v_c = (y_c / z_c) * fy_c + cy_c

# Round FIRST then check bounds
u_c = np.round(u_c).astype(int)
v_c = np.round(v_c).astype(int)

# Create new validity mask after rounding
valid = (u_c >= 0) & (u_c < width_d) & (v_c >= 0) & (v_c < height_d) & (z_c > 0)

# Apply final filter
u_c = u_c[valid]
v_c = v_c[valid]
transformed_points = transformed_points[valid]

# rotate 180 degrees around x-axis
R_flip = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
transformed_points = transformed_points @ R_flip.T

# Clip coordinates to ensure they stay within bounds (safety measure)
u_c = np.clip(u_c, 0, width_d-1)
v_c = np.clip(v_c, 0, height_d-1)

# Sample colors from color image
colors = color_image[v_c, u_c]

# Create point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(transformed_points)
pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

# Visualize and save
o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("aligned_pointcloud.ply", pcd)

True